In [1]:
import re
import requests 
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup

In [2]:
def parse_strlist(sl):
    clean = re.sub("[\[\],\s]","",sl)
    splitted = re.split("[\'\"]",clean)
    values_only = [s for s in splitted if s != '']
    return values_only

In [3]:
def get_graph_values(var_name, url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  scripts = soup.find_all('script')
  for script in scripts:
      if 'd = new Dygraph(document.getElementById("container")' in script.text:
          StrList = script.text
          StrList = '[[' + StrList.split('[[')[-1]
          StrList = StrList.split(']]')[0] +']]'
          StrList = StrList.replace("new Date(", '').replace(')','')
          dataList = parse_strlist(StrList)

  date = []
  value = []
  for each in dataList:
      if (dataList.index(each) % 2) == 0:
          date.append(each)
      else:
          value.append(each)

  df = pd.DataFrame(list(zip(date, value)), columns=["date",var_name])
  return df

In [4]:
url_dict = {'market_capitalisation' : 'https://bitinfocharts.com/comparison/bitcoin-marketcap.html',
            # number of bitcoins sent in USD
            'sent_usd' : 'https://bitinfocharts.com/comparison/bitcoin-sentinusd.html',
            # number of transactions
            'transactions' : 'https://bitinfocharts.com/comparison/bitcoin-transactions.html',
            # transaction value
            'avg_transaction_value' : 'https://bitinfocharts.com/comparison/bitcoin-transactionvalue.html',
            'median_transaction_value' : 'https://bitinfocharts.com/comparison/bitcoin-mediantransactionvalue.html',
            # transaction fee
            'avg_transaction_fee' : 'https://bitinfocharts.com/comparison/bitcoin-transactionfees.html',
            'median_transacation_fee' : 'https://bitinfocharts.com/comparison/bitcoin-median_transaction_fee.html',
            # average time between blocks
            'block_time' : 'https://bitinfocharts.com/comparison/bitcoin-confirmationtime.html',
            # average block size
            'block_size' : 'https://bitinfocharts.com/comparison/bitcoin-size.html',
            # average fee percentage in total block reward
            'fee' :'https://bitinfocharts.com/comparison/bitcoin-fee_to_reward.html',
            # average minimum difficulty
            'difficulty' : 'https://bitinfocharts.com/comparison/bitcoin-difficulty.html',
            # hash per second
            'hashrate' : 'https://bitinfocharts.com/comparison/bitcoin-hashrate.html',
            # profit as USD per day for 1 THash per second
            'mining_profitability' : 'https://bitinfocharts.com/comparison/bitcoin-mining_profitability.html#1y',
            # number of unique addresses
            'active_adresses' : 'https://bitinfocharts.com/comparison/bitcoin-activeaddresses.html',
            # percentage of total coins held by richest 100 addresses
            'top_addresses' : 'https://bitinfocharts.com/comparison/top100cap-btc.html',
            # number of tweets
            'tweets' : 'https://bitinfocharts.com/comparison/bitcoin-tweets.html',
            # number of searches on google
            'google_trends' : 'https://bitinfocharts.com/comparison/google_trends-btc.html',
            # average price
            'price' : 'https://bitinfocharts.com/comparison/bitcoin-price.html'
            }

In [5]:
bitinfo_df = pd.DataFrame(columns=['date'])

for item in url_dict.items():
  var_name = item[0]
  url = item[1]
  df = get_graph_values(var_name, url)
  bitinfo_df = bitinfo_df.merge(df, how='outer')

In [6]:
bitinfo_df['date'] = pd.to_datetime(bitinfo_df['date'])
bitinfo_df.head()

,date,market_capitalisation,sent_usd,transactions,avg_transaction_value,median_transaction_value,avg_transaction_fee,median_transacation_fee,block_time,block_size,fee,difficulty,hashrate,mining_profitability,active_adresses,top_addresses,tweets,google_trends,price
0,2010-07-17,169839,1193,235,2.924,0.248,1.05E-5,null,8.324,649.653,0.001,181.543,2775561176,154298,804,25.749,NaN,0.0287,0.0495
1,2010-07-18,249761,2612,248,6.237,0.363,0.000243,null,8.372,765.285,0.0163,181.543,1554461318,401834,917,25.782,NaN,0.0365,0.0726
2,2010-07-19,295981,4047,354,7.666,0.429,2.18E-5,null,8.276,756.04,0.00154,181.543,1551287489,481473,972,25.685,NaN,0.0213,0.0859
3,2010-07-20,270546,2341,413,3.94,5.105,null,null,7.956,984.707,null,181.543,1640430286,431831,906,25.602,NaN,0.0612,0.0783
4,2010-07-21,265900,2122,256,4.583,3.837,null,null,6.957,542.483,null,181.543,1723492810,460783,809,25.638,NaN,0.0538,0.0767


In [7]:
stocks_dict = {'swiss_franc' : 'CHFUSD=X',
               'euro' : 'EURUSD=X',
               'british_poun' : 'GBPUSD=X',
               'japanese_yen' : 'JPY=X',
               'chinese_yen' : 'CNY=X',
               'ftse' : '^FTSE',
               'dow_jones' : '^DJI',
               's&p500' : '^GSPC',
               'crude-oil' : 'CL=F',
               'nasdaq' : '^IXIC',
               'gold' : 'GC=F'
               }

In [8]:
tickers = list(stocks_dict.values())
df = yf.download(tickers, start='2010-01-01')

[*********************100%***********************]  11 of 11 completed


In [9]:
df.head()

Adj Close             ...        Volume              
            CHFUSD=X       CL=F  ...         ^GSPC         ^IXIC
Date                             ...                            
2010-01-01  0.967211        NaN  ...           NaN           NaN
2010-01-04  0.971723  81.510002  ...  3.991400e+09  1.931380e+09
2010-01-05  0.967399  81.769997  ...  2.491020e+09  2.367860e+09
2010-01-06  0.973141  83.180000  ...  4.972660e+09  2.253340e+09
2010-01-07  0.967492  82.660004  ...  5.270680e+09  2.270050e+09

[5 rows x 66 columns]

In [10]:
yahoo_df = df['Close']
yahoo_df.reset_index(inplace=True)
yahoo_df.head()

,Date,CHFUSD=X,CL=F,CNY=X,EURUSD=X,GBPUSD=X,GC=F,JPY=X,^DJI,^FTSE,^GSPC,^IXIC
0,2010-01-01,0.967211,NaN,NaN,1.438994,1.613294,NaN,92.919998,NaN,NaN,NaN,NaN
1,2010-01-04,0.971723,81.510002,6.8169,1.442398,1.611370,1117.699951,92.495003,10583.959961,5500.299805,1132.989990,2308.419922
2,2010-01-05,0.967399,81.769997,6.8170,1.436596,1.599079,1118.099976,91.584999,10572.019531,5522.500000,1136.520020,2308.709961
3,2010-01-06,0.973141,83.180000,6.8183,1.440403,1.602384,1135.900024,92.356003,10573.679688,5530.000000,1137.140015,2301.090088
4,2010-01-07,0.967492,82.660004,6.8178,1.431803,1.593397,1133.099976,93.389999,10606.860352,5526.700195,1141.689941,2300.050049


In [11]:
combined_df = pd.merge(bitinfo_df, yahoo_df, how ="outer", left_on='date', right_on='Date')
combined_df.drop(['Date'],axis=1, inplace=True)
combined_df = combined_df.sort_values('date')
combined_df

,date,market_capitalisation,sent_usd,transactions,avg_transaction_value,median_transaction_value,avg_transaction_fee,median_transacation_fee,block_time,block_size,fee,difficulty,hashrate,mining_profitability,active_adresses,top_addresses,tweets,google_trends,price,CHFUSD=X,CL=F,CNY=X,EURUSD=X,GBPUSD=X,GC=F,JPY=X,^DJI,^FTSE,^GSPC,^IXIC
4023,2009-01-03,NaN,NaN,null,NaN,NaN,NaN,NaN,NaN,204,NaN,1,null,NaN,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4024,2009-01-04,NaN,NaN,null,NaN,NaN,NaN,NaN,NaN,null,NaN,null,null,NaN,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4025,2009-01-05,NaN,NaN,null,NaN,NaN,NaN,NaN,NaN,null,NaN,null,null,NaN,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4026,2009-01-06,NaN,NaN,null,NaN,NaN,NaN,NaN,NaN,null,NaN,null,null,NaN,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4027,2009-01-07,NaN,NaN,null,NaN,NaN,NaN,NaN,NaN,null,NaN,null,null,NaN,null,null,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,2021-07-18,595880906407,7945136415,160978,231237,644.551,2.38,0.325,10,480527,1.324,13672594272814,9.7185384983432E+19,0.298,535632,15.22,75089,NaN,31763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4020,2021-07-19,585114953256,11001232041,226633,247413,719.669,2.614,0.267,8.471,557230,1.758,13672594272814,1.0143287645526E+20,0.333,738844,15.227,83081,NaN,31188,1.088696,66.419998,6.4782,1.181401,1.376387,1808.699951,109.874001,33962.039062,6844.399902,4258.490234,14274.980469
4021,2021-07-20,559789752886,12601597123,230387,235672,777.263,2.308,0.353,11.803,817866,2.285,13672594272814,1.021523953792E+20,0.228,710781,15.181,102358,NaN,29837,1.088732,67.419998,6.4894,1.179384,1.366998,1810.900024,109.533997,34511.988281,6881.100098,4323.060059,14498.879883
4022,2021-07-21,584867227348,14264112899,235535,248098,767.957,1.914,0.301,9.931,711925,1.572,13672594272814,9.393770657224E+19,0.306,712527,15.158,150305,NaN,31172,1.085505,NaN,6.4839,1.178411,1.363327,NaN,109.919998,34798.000000,6998.299805,4358.689941,14631.950195


In [12]:
combined_df.to_csv('bitcoin_btc.csv')